In [18]:
import pygsti
from pygsti.extras import idletomography as idt

n_qubits = 1
gates = ["Gi","Gx","Gy","Gcnot"]
max_lengths = [1,2,4,8]
pspec = pygsti.processors.QubitProcessorSpec(n_qubits, gates, geometry='line', nonstd_gate_unitaries={():1})

mdl_target = pygsti.models.create_crosstalk_free_model(pspec)
paulidicts = idt.determine_paulidicts(mdl_target)
idle_experiments = idt.make_idle_tomography_list(n_qubits, max_lengths, paulidicts, maxweight=1)
print(len(idle_experiments), "idle tomography experiments for %d qubits" % n_qubits)
from pygsti.baseobjs import Label
updated_ckt_list= []
for ckt in idle_experiments:
    new_ckt= ckt.copy(editable=True)
    for i,lbl in enumerate(ckt):
        if lbl == Label(()):
            new_ckt[i] = Label(('Gi',0))
    updated_ckt_list.append(new_ckt)
mdl_datagen = pygsti.models.create_crosstalk_free_model(pspec,
                                                        lindblad_error_coeffs={'Gi': {'HX': 0.01, 'SX': 0.01}})
# Error models! Random with right CP constraints from Taxonomy paper
ds = pygsti.data.simulate_data(mdl_datagen, updated_ckt_list, 100000, seed=8675309)


# hardcode pauli fidpairs!?!?
from pygsti.extras.idletomography.pauliobjs import NQPauliState
#oh_lawd = [(NQPauliState("X"), NQPauliState("X"))]
# "pauli_fidpairs":oh_lawd
huh = [(NQPauliState("X",(1,)), NQPauliState("X", (0,)))]

results = idt.do_idle_tomography(n_qubits, ds, max_lengths, paulidicts, maxweight=1, advanced_options={"jacobian mode":"together", 'pauli_fidpairs':huh}, idle_string='Gi:0')
results.pauli_fidpairs['samebasis']
results.pauli_fidpairs['diffbasis']

36 idle tomography experiments for 1 qubits


KeyError: 0

In [12]:
repr(results.pauli_fidpairs['samebasis'][0][0])

'State[+X]'

In [14]:
results.pauli_fidpairs['diffbasis']

[(State[+Z], State[+Y]), (State[+X], State[+Z]), (State[-Y], State[-X])]

In [15]:
results.pauli_fidpairs['samebasis']

[(State[+X], State[-X]),
 (State[-Y], State[+Y]),
 (State[+Z], State[+Z]),
 (State[-X], State[+X]),
 (State[+Y], State[-Y]),
 (State[-Z], State[-Z])]